# Analyzing statistics of Conv3x3 layers in pretrained models

### Setup:

`conda install -c conda-forge ipywidgets`

*OR*

`pip install ipywidgets` 

`jupyter nbextension enable --py widgetsnbextension`

*AND*

`conda install nb_conda_kernels`


In [2]:
import mxnet as mx
import gluoncv
from collections import namedtuple

In [3]:
model_name = 'ResNet50_v1d'

In [4]:
# download and load the pre-trained model
net = gluoncv.model_zoo.get_model(model_name, pretrained=True)

55464KB [00:05, 9386.15KB/s]                            


In [7]:
params = net.collect_params()

In [12]:
params

resnetv1d_ (
  Parameter resnetv1d_conv0_weight (shape=(32, 3, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter resnetv1d_batchnorm0_gamma (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter resnetv1d_batchnorm0_beta (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter resnetv1d_batchnorm0_running_mean (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter resnetv1d_batchnorm0_running_var (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter resnetv1d_conv1_weight (shape=(32, 32, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter resnetv1d_batchnorm1_gamma (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter resnetv1d_batchnorm1_beta (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter resnetv1d_batchnorm1_running_mean (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter resnetv1d_batchnorm1_running_var (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter resnetv1d_conv2_weight (shape=(64, 32, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter resnetv1d_bat

In [13]:
params.get('conv0_weight')

Parameter resnetv1d_conv0_weight (shape=(32, 3, 3, 3), dtype=<class 'numpy.float32'>)

In [15]:
[p for p in params if 'conv' and 'weight' in p]

['resnetv1d_conv0_weight',
 'resnetv1d_conv1_weight',
 'resnetv1d_conv2_weight',
 'resnetv1d_layers1_conv0_weight',
 'resnetv1d_layers1_conv1_weight',
 'resnetv1d_layers1_conv2_weight',
 'resnetv1d_down1_conv0_weight',
 'resnetv1d_layers1_conv3_weight',
 'resnetv1d_layers1_conv4_weight',
 'resnetv1d_layers1_conv5_weight',
 'resnetv1d_layers1_conv6_weight',
 'resnetv1d_layers1_conv7_weight',
 'resnetv1d_layers1_conv8_weight',
 'resnetv1d_layers2_conv0_weight',
 'resnetv1d_layers2_conv1_weight',
 'resnetv1d_layers2_conv2_weight',
 'resnetv1d_down2_conv0_weight',
 'resnetv1d_layers2_conv3_weight',
 'resnetv1d_layers2_conv4_weight',
 'resnetv1d_layers2_conv5_weight',
 'resnetv1d_layers2_conv6_weight',
 'resnetv1d_layers2_conv7_weight',
 'resnetv1d_layers2_conv8_weight',
 'resnetv1d_layers2_conv9_weight',
 'resnetv1d_layers2_conv10_weight',
 'resnetv1d_layers2_conv11_weight',
 'resnetv1d_layers3_conv0_weight',
 'resnetv1d_layers3_conv1_weight',
 'resnetv1d_layers3_conv2_weight',
 'resnetv1d